In [15]:
%pip install vaderSentiment
%pip install praw
%pip install python-dotenv
%pip install pandas
%pip install matplotlib
%pip install seaborn
import praw
from dotenv import load_dotenv
from datetime import datetime
import os
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


load_dotenv()


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note

True

In [16]:
# Initialize Reddit API
reddit = praw.Reddit(
    client_id=os.getenv("REDDIT_CLIENT_ID"),
    client_secret=os.getenv("REDDIT_CLIENT_SECRET"),
    user_agent=os.getenv("REDDIT_USER_AGENT"),
)
print(f"Reddit Read-Only: {reddit.read_only}")

Reddit Read-Only: True


In [24]:


# Define date range
start_date = datetime(2025, 1, 1)
end_date = datetime(2025, 11, 10)
start_timestamp = start_date.timestamp()
end_timestamp = end_date.timestamp()

In [25]:
# Define subreddit and query
politics_sub = reddit.subreddit("politics")
query = "Kamala Harris Trump election"

In [26]:
# Search and filter posts (Print)
for submission in politics_sub.search(query, sort="top", time_filter="month", limit=10):
    if start_timestamp <= submission.created_utc <= end_timestamp:
        print(f"Title: {submission.title}")
        print(f"Created at: {datetime.fromtimestamp(submission.created_utc)}")
        print(f"Score: {submission.score}")
        print(f"URL: {submission.url}\n")

Title: Just to Be Clear, No, Trump Can’t Be Elected President Again
Created at: 2025-03-31 08:19:49
Score: 22406
URL: https://www.rollingstone.com/politics/political-commentary/trump-serve-third-term-constitution-1235210225/

Title: Donald Trump Sends Election Message to Canadians: 'Cherished 51st State'
Created at: 2025-04-28 09:20:07
Score: 14579
URL: https://www.newsweek.com/trump-election-message-canada-51st-state-2065007

Title: Trump Order Includes Provision That Could Punish States For Not Ceding Authority Over Election Admin To DOJ
Created at: 2025-03-29 06:17:37
Score: 11865
URL: https://talkingpointsmemo.com/news/trump-order-includes-provision-that-could-punish-states-for-not-ceding-authority-over-election-admin-to-doj

Title: James Carville Fears Trump Will Declare Martial Law and Rig the Elections
Created at: 2025-04-09 11:17:56
Score: 11844
URL: https://www.thedailybeast.com/james-carville-fears-trump-will-declare-martial-law-and-rig-the-elections/

Title: Obama and Harris

In [27]:
# Search and filter posts (Save to csv)

posts_data = []

for submission in politics_sub.search(query, sort="top", time_filter="month", limit=10):
    if start_timestamp <= submission.created_utc <= end_timestamp:
        posts_data.append({
            'title': submission.title,
            'created_at': datetime.fromtimestamp(submission.created_utc),
            'score': submission.score,
            'url': submission.url
        })

In [28]:
df = pd.DataFrame(posts_data)
df.to_csv('data/reddit_posts_test.csv', index=False)
print(f"Saved {len(posts_data)} posts to reddit_posts.csv")

Saved 10 posts to reddit_posts.csv


In [29]:

# Sentiment Analyzer
analyzer = SentimentIntensityAnalyzer()

# Function to collect and analyze Reddit data
def collect_movie_data(movie_name, time_filter="month", limit=100):
    """
    Collects Reddit posts related to a movie, analyzes sentiment, and returns results.
    """
    sentiment_data = []
    query = f'"{movie_name}"'  # Search for the exact movie name
    print(f"Searching for posts about '{movie_name}'...")
    
    for submission in reddit.subreddit("all").search(query, sort="relevance", time_filter=time_filter, limit=limit):
        post_sentiment = analyzer.polarity_scores(submission.title + " " + (submission.selftext or ""))
        sentiment_data.append({
            "title": submission.title,
            "text": submission.selftext,
            "upvotes": submission.score,
            "comments": submission.num_comments,
            "created_utc": datetime.fromtimestamp(submission.created_utc),
            "sentiment_score": post_sentiment["compound"],
            "positive_score": post_sentiment["pos"],
            "negative_score": post_sentiment["neg"],
            "neutral_score": post_sentiment["neu"],
            "url": submission.url
        })
    
    print(f"Collected {len(sentiment_data)} posts for '{movie_name}'.")
    return sentiment_data

In [30]:
# Example Usage
movie_name = "Nosferatu"
data = collect_movie_data(movie_name, time_filter="month", limit=200)

# Display results
for entry in data:
    print(f"Title: {entry['title']}")
    print(f"Sentiment: {entry['sentiment_score']}, Upvotes: {entry['upvotes']}, Comments: {entry['comments']}")
    print(f"URL: {entry['url']}\n")

Searching for posts about 'Nosferatu'...
Collected 200 posts for 'Nosferatu'.
Title: These stackable Nosferatu mugs.
Sentiment: 0.0, Upvotes: 3432, Comments: 31
URL: https://i.redd.it/5ca95zj870we1.jpeg

Title: I still don't get why every clan seems to hate the Nosferatu in lore when they're consistently one of the most useful
Sentiment: -0.1298, Upvotes: 659, Comments: 144
URL: https://i.redd.it/jt0stzkv5zh41.png

Title: Suspect charged with setting Pa. Governor’s Residence on fire arrives at court
Sentiment: -0.6597, Upvotes: 36216, Comments: 3598
URL: https://i.redd.it/psjjh6bofvue1.jpeg

Title: NOSFERATU- Great behind the scenes pic
Sentiment: 0.7506, Upvotes: 1169, Comments: 42
URL: https://i.redd.it/5ddvca6krsve1.jpeg

Title: I have the hands of a 80 year old at 20
Sentiment: -0.1779, Upvotes: 5694, Comments: 1758
URL: https://www.reddit.com/gallery/1k07ysf

Title: Nosferatu (2024)
Sentiment: 0.0, Upvotes: 974, Comments: 34
URL: https://i.redd.it/yrwkkb5q68te1.jpeg

Title: Found 

In [ ]:
# Convert to DataFrame if not already
df = pd.DataFrame(data)

# Method 1: Group by Week
weekly_sentiment = (df.groupby(pd.Grouper(key='created_utc', freq='W'))
                   .agg({
                       'sentiment_score': 'mean',
                       'title': 'count'  # Count of posts per week
                   })
                   .reset_index())

print("\nWeekly Sentiment Analysis:")
for _, row in weekly_sentiment.iterrows():
    print(f"Week of {row['created_utc'].strftime('%Y-%m-%d')}:")
    print(f"  Average Sentiment: {row['sentiment_score']:.3f}")
    print(f"  Number of Posts: {row['title']}")
    print()